# Exploring industry returns in the Fama-French data-set

The [49 Industry Portfolios](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_49_ind_port.html) data-set has a series of daily returns of 49 different industry types. Here, we look at some of their statistical properties.

The documentation for the Fama-French data-set can be found [here](https://plutopy.readthedocs.io/en/latest/FamaFrench.html) and [here](https://shyams80.github.io/plutoR/docs/reference/FamaFrench-class.html)

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(lubridate)
library(reshape2)
library(PerformanceAnalytics)
library(ggrepel)
library(tbl2xts)

options("scipen"=999)
options(stringsAsFactors = FALSE)

source("config.R")
source("goofy/plot.common.R")

#initialize
famaFrench <- FamaFrench()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘lubridate’

The following object is masked from ‘package:b

In [2]:
startDt <- as.Date("1990-01-01")

#value (market-cap) weighted
valueWtd <- famaFrench$Industry49Daily() %>%
    filter(RET_TYPE == 'AVWRD' & TIME_STAMP >= startDt) %>%
    mutate(R = RET/100) %>%
    select(DATE = TIME_STAMP, KEY_ID, R) %>%
    collect() %>% 
    # the KEY_ID column has industry id's
    # we want them as column names
    mutate(group=1) %>%
    spread(KEY_ID, R) %>%
    select(-group) %>%
    tbl_xts()

#equal weighted
equalWtd <- famaFrench$Industry49Daily() %>%
    filter(RET_TYPE == 'AEWRD' & TIME_STAMP >= startDt) %>%
    mutate(R = RET/100) %>%
    select(DATE = TIME_STAMP, KEY_ID, R) %>%
    collect() %>% 
    # the KEY_ID column has industry id's
    # we want them as column names
    mutate(group=1) %>%
    spread(KEY_ID, R) %>%
    select(-group) %>%
    tbl_xts()

In [3]:
lookbackDays <- 220 * 5

rrValueWtd <- rollapply(valueWtd, lookbackDays, Return.cumulative, by.column = F)
rrEqualWtd <- rollapply(equalWtd, lookbackDays, Return.cumulative, by.column = F)

rrValueWtd <- na.omit(rrValueWtd)
rrEqualWtd <- na.omit(rrEqualWtd)

names(rrValueWtd) <- names(valueWtd)
names(rrEqualWtd) <- names(equalWtd)

In [4]:
minValWtd <- apply(rrValueWtd, 2, min)
minValWtd <- data.frame(minValWtd)

minEqlWtd <- apply(rrEqualWtd, 2, min)
minEqlWtd <- data.frame(minEqlWtd)

maxValWtd <- apply(rrValueWtd, 2, max)
maxValWtd <- data.frame(maxValWtd)

maxEqlWtd <- apply(rrEqualWtd, 2, max)
maxEqlWtd <- data.frame(maxEqlWtd)

sdValWtd <- apply(rrValueWtd, 2, sd)
sdValWtd <- data.frame(sdValWtd)
sdValWtd <- cbind(sdValWtd, row.names(sdValWtd))
print(sdValWtd)

sdEqlWtd <- apply(rrEqualWtd, 2, sd)
sdEqlWtd <- data.frame(sdEqlWtd)

temp <- merge(minValWtd, maxValWtd, by=0, all=T)
statsValDf <- merge.data.frame(temp, sdValWtd, by=0, all=T)
statsEqlDf <- merge.data.frame(merge.data.frame(minEqlWtd, maxEqlWtd, by=0, all=T), sdEqlWtd, by=0, all=T)

#print("MKT CAP WT")
#print(head(statsValDf))

#print("EQ WT")
#print(head(statsEqlDf))

       sdValWtd row.names(sdValWtd)
AERO  0.6732770                AERO
AGRIC 0.5828927               AGRIC
AUTOS 0.6150468               AUTOS
BANKS 0.8154830               BANKS
BEER  0.5572048                BEER
BLDMT 0.5667385               BLDMT
BOOKS 0.6729852               BOOKS
BOXES 0.4530478               BOXES
BUSSV 0.5353209               BUSSV
CHEMS 0.4673437               CHEMS
CHIPS 1.1536726               CHIPS
CLTHS 0.5133042               CLTHS
CNSTR 0.7403047               CNSTR
COAL  1.6672218                COAL
DRUGS 0.8628797               DRUGS
ELCEQ 0.8830644               ELCEQ
FABPR 0.5982059               FABPR
FIN   0.9356484                 FIN
FOOD  0.3957456                FOOD
FUN   0.7353736                 FUN
GOLD  0.6819889                GOLD
GUNS  0.7423928                GUNS
HARDW 1.3518820               HARDW
HLTH  0.4585224                HLTH
HSHLD 0.4467666               HSHLD
INSUR 0.6054176               INSUR
LABEQ 0.5885724             

Warning message in merge.data.frame(temp, sdValWtd, by = 0, all = T):
“column name ‘Row.names’ is duplicated in the result”Warning message in merge.data.frame(merge.data.frame(minEqlWtd, maxEqlWtd, by = 0, :
“column name ‘Row.names’ is duplicated in the result”

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)